In [ ]:
import tensorflow as tf
import numpy as np
from itertools import islice
import re
from tqdm import tqdm
import pickle
import time
import random

- movie_lines.txt
	- contains the actual text of each utterance
	- fields:
		- lineID
		- characterID (who uttered this phrase)
		- movieID
		- character name
		- text of the utterance

In [ ]:
movie_lines = open("cornell movie-dialogs corpus/movie_lines.txt").read().split('\n')
movie_lines[:5]

- movie_conversations.txt
	- the structure of the conversations
	- fields
		- characterID of the first character involved in the conversation
		- characterID of the second character involved in the conversation
		- movieID of the movie in which the conversation occurred
		- list of the utterances that make the conversation, in chronological 
			order: ['lineID1','lineID2',É,'lineIDN']
			has to be matched with movie_lines.txt to reconstruct the actual content

In [ ]:
movie_conv = open("cornell movie-dialogs corpus/movie_conversations.txt").read().split('\n')
movie_conv[:5]

In [ ]:
#lower too
indi_line = {}
for dia in movie_lines:
    indi_line[dia.split(' +++$+++ ')[0]] = dia.split(' +++$+++ ')[-1].lower()  

print("any 5 individual lines")
dict(islice(indi_line.items(), 5))

In [ ]:
conversation = []
for conv in movie_conv:
    i = []
    p = conv.split(' +++$+++ ')[-1]   
    for m in re.findall(r"[\w']+", p):
        i.append(m.split('\'')[1])
    conversation.append(i)
conversation[:5]   

In [ ]:
def special_char(word):
    word = re.sub(r"i'm", "i am", word)
    word = re.sub(r"you're", "you are", word)
    word = re.sub(r"don't", "do not", word)
    word = re.sub(r"you'd", "you would", word)
    word = re.sub(r"i've", "i have", word)
    word = re.sub(r"can't", "can not", word)
    word = re.sub(r"didn't", "did not", word)
    word = re.sub(r"aren't", "are not", word)
    word = re.sub(r"it's", "it is", word)
    word = re.sub(r"i'd", "i would", word)
    word = re.sub(r"isn't", "is not", word)
    word = re.sub(r"we'd", "we had", word)
    word = re.sub(r"won't", "will not", word)
    word = re.sub(r"you'll", "you will", word)
    word = re.sub(r"couldn't", "could not", word)
    word = re.sub(r"could've", "could have", word)
    word = re.sub(r"hasn't", "has not", word)
    word = re.sub(r"how's", "how is", word)
    word = re.sub(r"goin'", "going", word)
    
    
    
    return word

In [ ]:
'''Dealing with words that contain apostrophe using special_char method
   Also deleting any other symbol present'''

individual_line1 = {}
for key,line in indi_line.items():
    diag = ""
    for p in line.split(' '):
        diag+= special_char(p)+" "
        individual_line1[key]=diag.strip()

individual_line = {}
for key,line in individual_line1.items():
    diag = ""
    for p in line.split():
        diag+= re.sub('[^a-zA-Z]+', '', p)+" "
        
    individual_line[key]=diag.strip()

In [ ]:
dict(islice(individual_line.items(), 5))

In [ ]:
check_length = []
max_len = 0
for line in individual_line.values():
    count = 0
    for word in line.split():
        check_length.append(word)
        count+=1
    if count>max_len:
        max_len = count
        
check_length = set(check_length)
print("Vocabulary length",len(check_length))
print("Maximum movie line length",max_len)

### Since the length 555 is very large.


In [ ]:
max_len = 20
min_len = 5
final_conv = []
for conv in conversation:
    c = []
    for line in conv:
        if len(individual_line[line].split())<max_len and len(individual_line[line].split())>min_len:           
            c.append(individual_line[line])
    if(len(c)!=0):
        final_conv.append(c)

In [ ]:
final_conv[:2]

In [ ]:
word2idx = []

for conv in final_conv:
    for line in conv:
        for word in line.split():
            word2idx.append(word)
        
word2idx = sorted(set(word2idx))
print("Vocabulary length",len(word2idx))
word2idx[:5]

In [ ]:
idx2word = {}
for index,value in enumerate(word2idx):
    idx2word[index] = value

In [ ]:
index_conversation = []
for conv in tqdm(final_conv):
    c = []
    for line in conv:
        p = []
        for word in line.split():
            p.append(word2idx.index(word))
        c.append(p)
    index_conversation.append(c)
   


In [ ]:
word2idx.append('<PAD>')
idx2word[word2idx.index('<PAD>')] = '<PAD>'

word2idx.append('<EOS>')
idx2word[word2idx.index('<EOS>')] = '<EOS>'

In [ ]:
with open("word2idx.p", "wb") as encoded_pickle:
    pickle.dump(word2idx, encoded_pickle)
    
with open("idx2word.p", "wb") as encoded_pickle:
    pickle.dump(idx2word, encoded_pickle)
    
with open("index_conversation.p", "wb") as encoded_pickle:
    pickle.dump(index_conversation, encoded_pickle)

In [ ]:
word2idx = pickle.load(open('word2idx.p', 'rb'))
idx2word = pickle.load(open('idx2word.p', 'rb'))
index_conversation = pickle.load(open('index_conversation.p', 'rb'))

In [ ]:
print(final_conv[0], end = '\n\n')
print(index_conversation[0])

In [ ]:
indexed_question = []
indexed_answer = []

for conv in index_conversation:
    for i in range(len(conv)-1):
        indexed_question.append(conv[i])
        indexed_answer.append(conv[i+1])

In [ ]:
print("Complete first Conversation :", index_conversation[0],end='\n\n')
print("First two questions :",indexed_question[:2],end='\n\n')
print("First two answers :",indexed_answer[0:2])

In simple words, in a 'conversation', the first line is the question and the answer is the second line, similarly then the second line becomes the question and the third line becomes the answer and so on

In [ ]:
with open("indexed_question.p", "wb") as encoded_pickle:
    pickle.dump(indexed_question, encoded_pickle)
with open("indexed_answer.p", "wb") as encoded_pickle:
    pickle.dump(indexed_answer, encoded_pickle)    

In [ ]:
indexed_question = pickle.load(open('indexed_question.p', 'rb'))
indexed_answer = pickle.load(open('indexed_answer.p', 'rb'))

In [ ]:
final_indexed_ques = []
final_indexed_target = []
pad_id = word2idx.index('<PAD>')
eos_id = word2idx.index('<EOS>')
max_len = 23   #increasing the size due to the addition of PAD and EOS

for i in indexed_question:
    i = i + ([pad_id] * (max_len - len(i)))
    final_indexed_ques.append(i)
    

for i in indexed_answer:
    i = i + [eos_id] + ([pad_id] * (max_len - len(i) - 1 )) #minus 1 because eos_id is added
    final_indexed_target.append(i)

In [ ]:
with open("final_indexed_ques.p", "wb") as encoded_pickle:
    pickle.dump(final_indexed_ques, encoded_pickle)

with open("final_indexed_target.p", "wb") as encoded_pickle:
    pickle.dump(final_indexed_target, encoded_pickle) 

In [ ]:
final_indexed_ques = pickle.load(open('final_indexed_ques.p', 'rb'))
final_indexed_target = pickle.load(open('final_indexed_target.p', 'rb'))

In [ ]:
print('Final Indexed Question in words\n')
for word_in_index in final_indexed_ques[0]:
    print(idx2word[word_in_index], end=' ')

print("\nFinal Indexed Target")
for word_in_index in final_indexed_target[0]:
    print(idx2word[word_in_index], end=' ')
print("\n",final_indexed_target[0])

In [ ]:
'''data_split = [0.7,0.3]
total_length = len(final_indexed_ques)

split_lengths = [int(s*total_length) for s in data_split]
train_en_sequence, train_de_input, train_target = np.array(final_indexed_ques[:split_lengths[0]]), np.array(final_indexed_ans[:split_lengths[0]]), np.array(final_indexed_target[:split_lengths[0]])

test_en_sequence, test_de_input, test_target = np.array(final_indexed_ques[-split_lengths[1] : ]) , np.array(final_indexed_ans[-split_lengths[1]:]), np.array(final_indexed_target[-split_lengths[1]:])

'''

In [ ]:
#batch
batch_size = 15
train_en_sequence_batches = []
train_target_batches = []

for batch_index in range(len(final_indexed_ques)//batch_size):
    train_en_sequence_batches.append(final_indexed_ques[(batch_index*batch_size):(batch_index*batch_size+batch_size)])
    train_target_batches.append(final_indexed_target[(batch_index*batch_size):(batch_index*batch_size+batch_size)])
    
train_en_sequence_batches = np.array(train_en_sequence_batches)
train_target_batches = np.array(train_target_batches)

In [ ]:
np.save('train_en_sequence_batches.npy', train_en_sequence_batches)
np.save('train_target_batches.npy', train_target_batches)

In [ ]:
tar_batches = np.load('train_target_batches.npy')
en_seq_batches = np.load('train_en_sequence_batches.npy')
final_indexed_ques = pickle.load(open('final_indexed_ques.p', 'rb'))
final_indexed_target = pickle.load(open('final_indexed_target.p', 'rb'))

reverse_indexed_questions = []
for batch in en_seq_batches:
    bat = []
    for ques in batch:
        bat.append(ques[::-1])
    reverse_indexed_questions.append(bat)
reverse_indexed_questions = np.array(reverse_indexed_questions)


word2idx = pickle.load(open('word2idx.p', 'rb'))
idx2word = pickle.load(open('idx2word.p', 'rb'))
max_len = 23
batch_size = 15
embedding_size=450
num_layers=2
word2idx.append('<GO>')
vocab_size=len(word2idx)
epochs = 100

## Creating Model

In [ ]:
def encoding_layer(inputs, embedding_size, num_layers, vocab_size, seq_len_tensor):
    
    def cell(units, r):
        layer = tf.contrib.rnn.BasicLSTMCell(units)
        return tf.contrib.rnn.DropoutWrapper(layer, r)

    encoder_cell = tf.nn.rnn_cell.MultiRNNCell([cell(embedding_size, 0.5) for _ in range(num_layers)])

    encoder_embedings = tf.contrib.layers.embed_sequence(inputs, vocab_size, embedding_size)
    
    en_output, en_state = tf.nn.dynamic_rnn(encoder_cell, encoder_embedings, seq_len_tensor, dtype=tf.float32)
    
    return en_output, en_state

In [ ]:
def process_targets(targets, batch_size, word2idx):
     
    endings = tf.strided_slice(targets, [0, 0], [batch_size, -1], [1, 1]) 
    processed_tar = tf.concat([tf.fill([batch_size, 1], word2idx.index('<GO>')), endings], 1)
    return processed_tar

In [ ]:
def attention_mechanism(embedding_size, en_output, en_state, encoder_decoder_seq_len, batch_size):
    
    
    #https://skymind.ai/wiki/attention-mechanism-memory-network
    
    
    lstm_cell = tf.nn.rnn_cell.LSTMCell(embedding_size)      
    lstm_cell = tf.nn.rnn_cell.DropoutWrapper(lstm_cell, output_keep_prob = 0.5)
    
    attention = tf.contrib.seq2seq.BahdanauAttention(embedding_size, en_output, encoder_decoder_seq_len)
    
    decoder_cell = tf.contrib.seq2seq.AttentionWrapper(lstm_cell, attention, embedding_size/2)
    
    attention_zero = decoder_cell.zero_state(batch_size=batch_size, dtype=tf.float32)
    enc_state_new = attention_zero.clone(cell_state=en_state[-1])
    
    return decoder_cell, enc_state_new

In [ ]:
def decoding_layer(batch_size, embedding_size, dec_inputs, vocab_size, decoder_cell, enc_state_new, seq_len_tensor, max_seq_len):
    
    embedding_layer = tf.Variable(tf.random_uniform([vocab_size, embedding_size]))
    embedding = tf.nn.embedding_lookup(embedding_layer, dec_inputs)
    
    output_layer = tf.layers.Dense(vocab_size, kernel_initializer=tf.truncated_normal_initializer(0.0, 0.1))
    
    
    with tf.variable_scope('decoder'):
        train_helper = tf.contrib.seq2seq.TrainingHelper(embedding, seq_len_tensor)
        
        train_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, 
                                                        train_helper, 
                                                        enc_state_new, 
                                                        output_layer)

        train_dec_outputs, _, _ = tf.contrib.seq2seq.dynamic_decode(train_decoder, impute_finished=True, maximum_iterations=max_seq_len)
        
        
    with tf.variable_scope('decoder', reuse=True):
        
        starting_id_vec = tf.tile(tf.constant([word2idx.index('<GO>')], dtype=tf.int32), [batch_size], name='starting_id_vec')
        
        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embedding_layer, 
                                                                    starting_id_vec, 
                                                                    word2idx.index('<EOS>'))
        
        inference_decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell,
                                                            inference_helper, 
                                                            enc_state_new, 
                                                            output_layer)
        
        
        inference_dec_output, _, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder, impute_finished=True, maximum_iterations=max_seq_len)
        
    return train_dec_outputs, inference_dec_output

In [ ]:
def loss_optimizer(train_outputs, encoder_decoder_seq_len, max_seq_len, targets, learning_rate=0.0001, clip_rate=4):
    
    logits = tf.identity(train_outputs.rnn_output)
    
    mask_weights = tf.sequence_mask(encoder_decoder_seq_len, max_seq_len, dtype=tf.float32)
    
    with tf.variable_scope('lo_op'):
        
        loss = tf.contrib.seq2seq.sequence_loss(logits, targets, mask_weights)
        
        opt = tf.train.AdamOptimizer(learning_rate)

        gradients = tf.gradients(loss, tf.trainable_variables())
        clipped_grads, _ = tf.clip_by_global_norm(gradients, clip_rate)
        traiend_opt = opt.apply_gradients(zip(clipped_grads, tf.trainable_variables()))
        
    return loss, traiend_opt

In [ ]:
tf.reset_default_graph()

inputs =  tf.placeholder(tf.int32, [None, None])
targets = tf.placeholder(tf.int32, [None, None])
encoder_decoder_seq_len = tf.placeholder(tf.int32, (None,))
max_seq_len = tf.reduce_max(encoder_decoder_seq_len, name='max_seq_len')

en_output, en_state = encoding_layer(inputs, embedding_size, num_layers, vocab_size, encoder_decoder_seq_len)

dec_inputs = process_targets(targets, batch_size, word2idx)

decoder_cell, enc_state_new = attention_mechanism(embedding_size, en_output, en_state, encoder_decoder_seq_len, batch_size)

train_outputs, inference_outputs = decoding_layer(batch_size, embedding_size, dec_inputs, vocab_size, decoder_cell, enc_state_new, encoder_decoder_seq_len, max_seq_len)

predictions  = tf.identity(inference_outputs.sample_id, name='preds')

loss, opt = loss_optimizer(train_outputs, encoder_decoder_seq_len, max_seq_len, targets, learning_rate=0.0001, clip_rate=4)

## Training

In [ ]:
saver = tf.train.Saver()


with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state('checkpoint/')

    if ckpt and ckpt.model_checkpoint_path:
        module_file = tf.train.latest_checkpoint('checkpoint/')
        saver.restore(sess, module_file)
    else:
        sess.run(tf.global_variables_initializer())

    try:
        total_loss = 0

        display_counter = 700

        for epoch in range(epochs):
            batch_index = 0 
            starting_time = time.time()
            
            for x_bat, y_bat in zip(reverse_indexed_questions, tar_batches):

                    
                    batch_index+=1

                    _, loss_val, ppp = sess.run([opt, loss, predictions], {inputs:x_bat, targets:y_bat,encoder_decoder_seq_len:[len(x_bat[0])]*batch_size})

                    total_loss+=loss_val

                    if batch_index%display_counter==0:
                        batch_time = time.time() - starting_time
                        print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'.format(epoch+1, epochs, batch_index, len(en_seq_batches), total_loss/display_counter, batch_time))
                        saver.save(sess, 'checkpoint/model')
                        starting_time = time.time()
                        total_loss=0

    except KeyboardInterrupt:
        print('Interrupted by user')
        
        
        
        
    
                     
    

## Predicting

In [ ]:
saver = tf.train.Saver()


with tf.Session() as sess:
    
    module_file = tf.train.latest_checkpoint('checkpoint/')
    saver.restore(sess, module_file)
    
    
    rand_ques_no = random.randint(0,len(final_indexed_ques))
    rand_ques = final_indexed_ques[rand_ques_no]
    
    
    
    print('\nQuestion:',end=" ")
    for id_ in rand_ques:
        if idx2word[id_]=='<PAD>':
            break
        print(idx2word[id_],end=" ")
    
    
    
    fake_bat = np.zeros((batch_size, max_len))
    fake_bat[0] = ques
    en_dc_len = np.ones(batch_size, dtype=np.int32)
    en_dc_len[0] = max_len    #determines the length of output for the fake_bat[0], which is nothing but a question
    
    
    pred = sess.run([predictions], feed_dict={inputs: fake_bat, encoder_decoder_seq_len:en_dc_len})[0][0]
    
    print('\nAnswer:',end=" ")
    for id_ in pred:
        if idx2word[id_]=='<EOS>':
            break
        print(idx2word[id_],end=" ")
